# We Read The Data from SQLite file
#### Data is downloaded from kaggle <a href="https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews?resource=download">Link</a>

In [2]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string


In [3]:
connection=sql.connect('../Data Sets/database.sqlite') #This line opens a connection to my D

In [4]:
print(connection)

In [5]:
pd_data=pd.read_sql_query("""
select * 
from Reviews
where Score != 3
""",connection) #Here we get the only positive or negative data

In [6]:
print(pd_data.head())

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [7]:
pd_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [8]:
def sep(x):
    if x<3:
        return 'Negative'
    else :
        return "Positive"

In [9]:
actual_score=pd_data['Score']

In [10]:
print(len(actual_score))

525814


In [11]:
pd_data.Score.isnull().sum()

0

In [12]:
#as there is not any NAN value exist in pd_data so we use .apply in place of .map by considering of getting a pandas data frame
scoreCard=pd_data.Score
scoreCard=scoreCard.map(sep)

In [13]:
scoreCard

0         Positive
1         Negative
2         Positive
3         Negative
4         Positive
            ...   
525809    Positive
525810    Negative
525811    Positive
525812    Positive
525813    Positive
Name: Score, Length: 525814, dtype: object

In [14]:
pd_data.Score=scoreCard

In [15]:
pd_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


# Data Cleaning: Data Duplication



In [16]:
dup=pd.read_sql_query("""
select *
from Reviews
where Score!=3 and UserID= "A395BORC6FGVXV"
""",connection)

In [17]:
dup

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
1,136304,B002Y7526Y,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
2,544173,B000U9WZ54,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...


# Deduping

In [18]:
shorted_data=pd_data.sort_values('ProductId',axis=0,ascending=True,kind='mergesort')

In [22]:
print(shorted_data.shape)

(525814, 10)


In [23]:
final=shorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep='first',inplace=False)
final.shape

(364173, 10)

In [26]:
(final.shape[0]*1.0/shorted_data.shape[0])*100
#Here we get that only 69% of the data are orignal rest of the others are duplicates

69.25890143662969

In [29]:
helpful_data=pd.read_sql_query("""
select *
from Reviews
where HelpfulnessNumerator>HelpfulnessDenominator
""",connection)
helpful_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
1,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


In [30]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138676,150493,0006641040,AMX0PJKV4PPNJ,"E. R. Bird ""Ramseelbird""",71,72,Positive,1096416000,Read it once. Read it twice. Reading Chicken S...,"These days, when a person says, ""chicken soup""..."
138677,150494,0006641040,AYZ0PR5QZROD1,Mother of 3 girls,3,3,Positive,1173312000,Family favorite,All of my children love this book. My first g...
138678,150496,0006641040,A3KKR87BJ0C595,"Gretchen Goodfellow ""Lover of children's lit""",3,3,Positive,1111363200,"You'll use it once, you'll use it twice",One of my earliest memories is of this book. ...
138679,150497,0006641040,A1HKYQOFC8ZZCH,"Maria Apolloni ""lanarossa""",2,2,Negative,1334707200,"The story is great, the softcover book is disa...",I give five stars to the Maurice Sendak story....
138680,150498,0006641040,A3SJWISOCP31TR,R. J. Wells,2,2,Positive,1176336000,A Gem of a Book,This is a wonderful little book. I loved it 40...


In [32]:
final.shape

(364171, 10)

In [34]:
final.Score.value_counts() #Used to count the values of unique numbers in the data set

Positive    307061
Negative     57110
Name: Score, dtype: int64